In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Embedding, Bidirectional, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [14]:
tokenizer = Tokenizer()

In [15]:
with open("lanigan ball.txt", encoding="utf-8") as conteudo:
    data = conteudo.read()

corpus = data.lower().split("\n")

In [16]:
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [17]:
input_sequences = []

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0] # tokeniza a lista

    for i in range(1, len(token_list)): # cria n-gramas
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
input_sequences

[[3, 2],
 [3, 2, 46],
 [3, 2, 46, 15],
 [3, 2, 46, 15, 47],
 [3, 2, 46, 15, 47, 48],
 [3, 2, 46, 15, 47, 48, 49],
 [3, 2, 46, 15, 47, 48, 49, 50],
 [3, 2, 46, 15, 47, 48, 49, 50, 51],
 [3, 2, 46, 15, 47, 48, 49, 50, 51, 23],
 [3, 2, 46, 15, 47, 48, 49, 50, 51, 23, 32],
 [3, 2, 46, 15, 47, 48, 49, 50, 51, 23, 32, 24],
 [3, 2, 46, 15, 47, 48, 49, 50, 51, 23, 32, 24, 52],
 [3, 2, 46, 15, 47, 48, 49, 50, 51, 23, 32, 24, 52, 4],
 [3, 2, 46, 15, 47, 48, 49, 50, 51, 23, 32, 24, 52, 4, 53],
 [16, 54],
 [16, 54, 55],
 [16, 54, 55, 56],
 [16, 54, 55, 56, 33],
 [16, 54, 55, 56, 33, 4],
 [16, 54, 55, 56, 33, 4, 57],
 [16, 54, 55, 56, 33, 4, 57, 17],
 [16, 54, 55, 56, 33, 4, 57, 17, 58],
 [16, 54, 55, 56, 33, 4, 57, 17, 58, 33],
 [16, 54, 55, 56, 33, 4, 57, 17, 58, 33, 4],
 [16, 54, 55, 56, 33, 4, 57, 17, 58, 33, 4, 59],
 [16, 54, 55, 56, 33, 4, 57, 17, 58, 33, 4, 59, 1],
 [16, 54, 55, 56, 33, 4, 57, 17, 58, 33, 4, 59, 1, 60],
 [16, 54, 55, 56, 33, 4, 57, 17, 58, 33, 4, 59, 1, 60, 61],
 [16, 54, 55

In [18]:
max_sequence_length = max([len(x) for x in input_sequences])

In [19]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding="pre"))

In [20]:
# slicing
xs = input_sequences[:, :-1]
labels = input_sequences[:, -1]

In [21]:
# one-hot encoding
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [22]:
model = Sequential()
model.add(Embedding(total_words, 345, input_length=max_sequence_length-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation="softmax"))
adam = Adam(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])
history = model.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/100
10/10 [==============================] - 4s 46ms/step - loss: 5.2333 - accuracy: 0.0411
Epoch 2/100
10/10 [==============================] - 1s 54ms/step - loss: 4.6321 - accuracy: 0.0791
Epoch 3/100
10/10 [==============================] - 0s 43ms/step - loss: 3.7145 - accuracy: 0.1487
Epoch 4/100
10/10 [==============================] - 0s 35ms/step - loss: 2.6912 - accuracy: 0.2753
Epoch 5/100
10/10 [==============================] - 0s 33ms/step - loss: 1.7559 - accuracy: 0.4873
Epoch 6/100
10/10 [==============================] - 0s 33ms/step - loss: 1.0875 - accuracy: 0.6994
Epoch 7/100
10/10 [==============================] - 0s 37ms/step - loss: 0.6365 - accuracy: 0.8418
Epoch 8/100
10/10 [==============================] - 0s 33ms/step - loss: 0.3045 - accuracy: 0.9335
Epoch 9/100
10/10 [==============================] - 0s 36ms/step - loss: 0.1905 - accuracy: 0.9589
Epoch 10/100
10/10 [==============================] - 0s 41ms/step - loss: 0.1120 - accuracy: 0.9684

In [ ]:
# houve uma precisão bem maior com as alterações práticas
seed_text = 'I made a poetry machine'
next_words = 20

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding="pre")
    predicted = model.predict(token_list, verbose=0)

    # index da maior probabilidade
    predicted_index = np.argmax(predicted, axis=-1)[0]

    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break

    # atualiza o texto com a nova palavra prevista
    seed_text += " " + output_word

print(seed_text)

I made a poetry machine town of athy one jeremy lanigan battered away till he hadn’t a shilling shilling shilling ground ground ground ground ground
